In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
import math as math
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
root.gStyle.SetOptFit(1111)
%jsroot on
%rm output/deltas/*.png

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
sim = 0
Min_Entries = 10
N_files = 1
Do_save = 1

In [3]:
N_hists = 6
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan,root.kTeal]
file_path="input/"
file_names="my-m_ee_Run14AuAu_92nd_new_19900_1065runs" 
#m_ee_Run14AuAu_91st_new_19891_1061runs m_ee_Run14AuAu_92nd_new_19900_799runs
#m_ee_Run14AuAu_84th_new_19850_337runs m_ee_Run14AuAu_83rd_new_19842_1065runs m_ee_Run14AuAu_85th_new_19858_120runs m_ee_Run14AuAu_86th_new_19862_872runs m_ee_Run14AuAu_87th_new_19865_518runs
hist_select_3D_names = ["DCA12_hist_0","DCA12_hist_1","DCA12_hist_2","DCA12_hist_3","DCA12_hist_4","hist_dca_x","hist_dca_y"]
N_hists = len(hist_select_3D_names)

In [4]:
if sim :
    file_path="../sim/output/Newembed/dca/" 
    file_names="piminus_50M_new_v7"#piminus_50M_v7 piminus_50M_v8 piminus_50M_v9
    N_hists = N_hists - 2

In [5]:
hists_read = []

infile = root.TFile.Open(file_path+file_names+f".root", "read")
for ihist in range(N_hists):
    hists_read.append(infile.Get(hist_select_3D_names[ihist]))
    hists_read[-1].SetDirectory(root.nullptr)
infile.Close()

In [6]:
z_edges = [-12+3*i for i in range(9)]
z_means = [-10.5+3*i for i in range(8)]
z_edges = array('d', z_edges)
z_means = array('d', z_means)
phi_edges = [-1.5 + 0.30*i for i in range(21)]
phi_means = [-1.35 + 0.30*i for i in range(20)]
phi_edges = array('d', phi_edges)
phi_means = array('d', phi_means)
def get_phi_zed_bin(in_value):
    if in_value > 179: in_value -= 180
    zed = int(in_value / 20)
    phi = int((in_value - zed * 20) )
    return zed, phi
def get_phi_value(phi_bin):
    return phi_means[phi_bin] if phi_bin >= 0 and phi_bin < len(phi_means) else 0
def get_zed_value(zed_bin):
    return z_means[zed_bin] if zed_bin >= 0 and zed_bin < len(z_means) else 0

In [7]:
def copycat_hist(hist,seed=0):
    seed += root.gRandom.Integer(1000)
    new_hist = root.TH3D(hist.GetName()+f"{seed}", hist.GetTitle()+f"{seed}", hist.GetNbinsX(), hist.GetXaxis().GetBinLowEdge(1), hist.GetXaxis().GetBinUpEdge(hist.GetNbinsX()),\
        hist.GetNbinsY(), hist.GetYaxis().GetBinLowEdge(1), hist.GetYaxis().GetBinUpEdge(hist.GetNbinsY()),\
        hist.GetNbinsZ(), hist.GetZaxis().GetBinLowEdge(1), hist.GetZaxis().GetBinUpEdge(hist.GetNbinsZ()))
    for ix in range(1, hist.GetNbinsX()+1):
        for iy in range(1, hist.GetNbinsY()+1):
            for iz in range(1, hist.GetNbinsZ()+1):
                new_hist.SetBinContent(ix, iy, iz, hist.GetBinContent(ix, iy, iz))
    return new_hist

In [8]:
ihits = 1
in_hist = hists_read[ihits]
c1 = root.TCanvas("c1", "c1", 1600, 1000)
c1.Divide(3,2)
local_bin_endges = [[1,100],[21,30],[51,60],[25,25],[55,55]]
sigmas = []
for i in range(5):
    if sim: continue
    c1.cd(i+1)
    hist_proj = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj{i}", 66, 101, local_bin_endges[i][0], local_bin_endges[i][1])
    #hist_proj.Draw("")
    hist_proj1 = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj{i}_1", 1, 35, local_bin_endges[i][0], local_bin_endges[i][1])
    hist_proj1.Draw("")
    #hist_proj.Fit("gaus", "Q","",-150,150)
    hist_proj1.Fit("gaus", "Q","",-150,150)
    sigmas.append([hist_proj1.GetFunction("gaus").GetParameter(1), hist_proj1.GetFunction("gaus").GetParameter(2)])

c1.cd(6)
hist_projy = in_hist.ProjectionY(in_hist.GetName()+"hist_projy", 21, 30, 1, 160)
#hist_projy.Draw("")

c1.Draw()

In [19]:
ihits = 1
in_hist = hists_read[ihits]
c1 = root.TCanvas("c1", "c1", 900, 900)
local_bin_endges = [[1,100],[21,30],[51,60],[25,25],[55,55]]
sigmas = []
legends = []
for i in range(1,2):
    if sim: continue
    hist_proj = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj{i}", 66, 101, local_bin_endges[i][0], local_bin_endges[i][1])
    #hist_proj.Draw("")
    hist_proj1 = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj{i}_1", 1, 35, local_bin_endges[i][0], local_bin_endges[i][1])
    for ibin in range(1, hist_proj1.GetNbinsX()+1):
        hist_proj1.SetBinError(ibin, hist_proj1.GetBinContent(ibin) ** 0.5/1000)
        hist_proj1.SetBinContent(ibin, hist_proj1.GetBinContent(ibin) /1000)
    Format_Hist_total(hist_proj1,"DCA_{T} [#mum]","dN/DCA_{T} [10^{3} per 20 #mum]",0.15,0.15,0.01,0.01,0.05,0.05,21,1,colors[0],3,colors[0],1.2,1.44)
    hist_proj1.GetXaxis().SetNdivisions(505)
    #root.gStyle.SetOptFit(1111)
    hist_proj1.Draw("p")
    #hist_proj.Fit("gaus", "Q","",-150,150)
    hist_proj1.Fit("gaus", "Q","",-120,120-5)
    sigmas.append([hist_proj1.GetFunction("gaus").GetParameter(1), hist_proj1.GetFunction("gaus").GetParameter(2)])
    errors = [hist_proj1.GetFunction("gaus").GetParError(1), hist_proj1.GetFunction("gaus").GetParError(2)]
    hist_proj1.GetFunction("gaus").SetLineColor(colors[3])
    hist_proj1.GetFunction("gaus").SetLineWidth(3)
    legends.append(root.TLegend(0.55, 0.7, 0.95, 0.95))
    Format_Legend(legends[-1], 0.035, 22)
    chi2 = hist_proj1.GetFunction("gaus").GetChisquare()
    ndf = hist_proj1.GetFunction("gaus").GetNDF()
    legends[-1].AddEntry(0, f"chi2/ndf = {chi2:.1f}"+"/"+f"{ndf:.1f}", "")
    legends[-1].AddEntry(0, f"constant = {hist_proj1.GetFunction('gaus').GetParameter(0)*1000:.0f} #pm {hist_proj1.GetFunction('gaus').GetParError(0)*1000:.0f}", "")
    legends[-1].AddEntry(0, f"mean = {sigmas[-1][0]:.2f} #pm {errors[0]:.2f} #mum", "")
    legends[-1].AddEntry(0, f"#sigma = {sigmas[-1][1]:.2f} #pm {errors[1]:.2f} #mum", "")
    legends[-1].Draw()
    legends.append(root.TLegend(0.17, 0.7, 0.5, 0.95,"Au+Au 200 GeV"))
    Format_Legend(legends[-1], 0.04, 22)
    legends[-1].AddEntry(hist_proj1, "h^{+} p_{T} > 1.5 GeV", "p")
    legends[-1].AddEntry(hist_proj1.GetFunction("gaus"), "gaus fit", "l")
    legends[-1].Draw()

hist_projy = in_hist.ProjectionY(in_hist.GetName()+"hist_projy", 21, 30, 1, 160)
#hist_projy.Draw("")

c1.Draw()
#c1.SaveAs("output/dca_hadron_peak_fit.pdf")

In [10]:
print("Sigma values for the histograms:")
for i, sigma in enumerate(sigmas):
    print(f"Histogram {i+1}: {sigma[0]:.3f}, {sigma[1]:.3f}")

Sigma values for the histograms:
Histogram 1: -2.417, 59.829


In [11]:
ihits = 1
pt = 15
in_hist = copycat_hist(hists_read[ihits])
for ibinx in range(in_hist.GetNbinsX()):
    for ibiny in range(in_hist.GetNbinsY()):
        for ibinz in range(in_hist.GetNbinsZ()):
            if ibiny<50+pt and ibiny>50-pt:
                in_hist.SetBinContent(ibinx, ibiny, ibinz, 0)
hist_proj = in_hist.Project3D("xz")

c1 = root.TCanvas("c1", "c1", 800, 600)
profile_x = hist_proj.ProfileX(hist_proj.GetName()+"profileX")
profile_x.Rebin(1)
#profile_x.Fit(fit_func, "Q", "", 50, 60)
profile_x.Draw("")
c1.Draw()


In [12]:
if False :
    x_offsets = array("d")
    x_abscissas = array("d")
    x_offsets_errors = array("d")
    x_arrays = {}
    for ibinx in range(profile_x.GetNbinsX()):
        x_offsets.append(profile_x.GetBinContent(ibinx+1))
        x_offsets_errors.append(profile_x.GetBinError(ibinx+1))
        x_abscissas.append(profile_x.GetBinCenter(ibinx+1))
    for i, (xx, yy, zz) in enumerate(zip(x_abscissas, x_offsets, x_offsets_errors)):
        z_bin, phi_bin = get_phi_zed_bin(xx)
        phi = get_phi_value(phi_bin)
        if z_bin not in x_arrays:
            x_arrays[z_bin] = [array('d'),array('d'),array('d'),array('d')]
        x_arrays[z_bin][0].append((yy))
        x_arrays[z_bin][1].append((phi))
        x_arrays[z_bin][2].append((zz))
        x_arrays[z_bin][3].append(0.15)
    
    print("x_arrays", x_arrays)

In [13]:
if True :
    x_offsets = array("d")
    x_abscissas = array("d")
    x_offsets_errors = array("d")
    x_arrays = {}
    all_projections = []
    for ibinx in range(profile_x.GetNbinsX()):
        local_mean = in_hist.GetBinContent(ibinx+1)
        local_projection = in_hist.ProjectionX(in_hist.GetName()+f"hist_proj_z_{ibinx}", 1, 101, ibinx+1, ibinx+1)
        local_projection.Fit("gaus", "Q","",local_mean-120,local_mean+120)
        all_projections.append(local_projection)
        if local_projection.GetEntries() > 500 and local_projection.GetFunction("gaus"):
            c1 = root.TCanvas("c1", "c1", 800, 600)
            x_offsets.append(local_projection.GetFunction("gaus").GetParameter(1))
            x_offsets_errors.append(2000./local_projection.GetFunction("gaus").Integral(-120, 120) **0.5 )
            local_projection.Draw("")
            c1.SaveAs(f"output/deltas/fit_zed_{ibinx}.png")
        else:
            x_offsets.append(0)
            x_offsets_errors.append(0)
        x_abscissas.append(profile_x.GetBinCenter(ibinx+1))
    for i, (xx, yy, zz) in enumerate(zip(x_abscissas, x_offsets, x_offsets_errors)):
        z_bin, phi_bin = get_phi_zed_bin(xx)
        phi = get_phi_value(phi_bin)
        if z_bin not in x_arrays:
            x_arrays[z_bin] = [array('d'),array('d'),array('d'),array('d')]
        x_arrays[z_bin][0].append((yy))
        x_arrays[z_bin][1].append((phi))
        x_arrays[z_bin][2].append((zz))
        x_arrays[z_bin][3].append(0.15)
    
    print("x_arrays", x_arrays)

x_arrays {0: [array('d', [0.0, 0.0, 0.0, -4.691687901055727, -1.398129055391826, 3.395364831301271, 7.1643110376624035, 10.846347096137984, -15.905373572734922, 0.0, 0.0, 0.0, 18.237549160718935, -10.909729228181124, -3.3412560356587795, 8.742008603760047, 15.465905359169794, 15.566864437876706, 0.0, 0.0]), array('d', [-1.35, -1.05, -0.7500000000000001, -0.4500000000000002, -0.15000000000000013, 0.1499999999999999, 0.44999999999999973, 0.75, 1.0499999999999998, 1.3499999999999996, 1.65, 1.9499999999999997, 2.2499999999999996, 2.55, 2.85, 3.15, 3.4499999999999997, 3.7499999999999996, 4.049999999999999, 4.35]), array('d', [0.0, 0.0, 0.0, 10.150426955732142, 8.570217900368867, 8.990550539045156, 5.701779020950861, 6.913951788887571, 14.517831619400814, 0.0, 0.0, 0.0, 16.554948572016098, 9.536770579211751, 7.414748100228255, 4.444339680040964, 6.149014447684823, 16.34735836169181, 0.0, 0.0]), array('d', [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.

In [14]:
if False:
    for iz in x_arrays:
        arr = x_arrays[iz][0]
        arr_errors = x_arrays[iz][2]
        n = len(arr)
        # Find indices of valid (non -200) entries in each region
        valid_indices_low = [i for i, v in enumerate(arr) if v != -200 and i < 10]
        valid_indices_high = [i for i, v in enumerate(arr) if v != -200 and i >= 10]
        for i in range(n):
            if arr[i] == -200:
                if i < 10 and valid_indices_low:
                    closest = min(valid_indices_low, key=lambda x: abs(x - i))
                    arr[i] = arr[closest]
                    arr_errors[i] = arr_errors[closest]
                elif i >= 10 and valid_indices_high:
                    closest = min(valid_indices_high, key=lambda x: abs(x - i))
                    arr[i] = arr[closest]
                    arr_errors[i] = arr_errors[closest]

In [15]:
c3 = root.TCanvas("c3", "c3", 1600, 1600)
c3.Divide(2, 4)
graphs_x = []
graphs_y = []
for iz in range(len(z_means)):
    c3.cd(iz+1)
    if iz not in x_arrays: continue
    graph_x = root.TGraphErrors(len(x_arrays[iz][0]), x_arrays[iz][1], x_arrays[iz][0], x_arrays[iz][3], x_arrays[iz][2])
    graph_x.SetTitle(f"Z={get_zed_value(iz)}")
    graph_x.GetXaxis().SetTitle("Phi")
    graph_x.GetYaxis().SetTitle("DCA X/Y")
    graph_x.SetMarkerStyle(20)
    graph_x.SetMarkerColor(colors[iz])
    graphs_x.append(graph_x)
    graph_x.SetMaximum(50)
    graph_x.SetMinimum(-50)
    graph_x.Draw("AP")
c3.Draw()

In [16]:
if False:
    for iz in x_arrays:
        arr = x_arrays[iz][0]
        n = len(arr)
        # Find indices of valid (non -200) entries
        valid_indices = [i for i, v in enumerate(arr) if v != -200]
        for i in range(n):
            if arr[i] == -200:
                # Find closest valid index
                if valid_indices:
                    closest = min(valid_indices, key=lambda x: abs(x - i))
                    arr[i] = arr[closest]

In [17]:
if True:
    print(f"const double offsets_dca[{len(x_arrays)}][{len(x_arrays[0][0])}] =" + " {")
    for iz in range(len(x_arrays)):
        if iz > 0: print(",")
        print("    {", end="")
        for i in range(len(x_arrays[iz][0])):
            if i > 0: print(", ", end="")
            print(f"{x_arrays[iz][0][i]/10000:.3g}", end="")
        print("}", end="")
    print("\n};")

const double offsets_dca[8][20] = {
    {0, 0, 0, -0.000469, -0.00014, 0.00034, 0.000716, 0.00108, -0.00159, 0, 0, 0, 0.00182, -0.00109, -0.000334, 0.000874, 0.00155, 0.00156, 0, 0},
    {0, 0, 0.00318, -0.000358, -0.00013, 0.000123, 1.24e-05, 0.000339, -0.00241, 0, 0, 0, 0.00223, -0.00136, -0.000259, 0.000121, 0.000655, 0.000278, 0, 0},
    {0, 0, 0.00345, -0.000751, 0.000436, 4.4e-05, -8.92e-05, 6.67e-05, -0.000462, 0, 0, 0, -0.00224, -0.00128, 0.00015, -7.76e-05, -0.000541, 0.000287, 0, 0},
    {0, 0, 0.00174, -0.000432, 0.00084, 5.29e-06, -0.000381, -0.000196, -0.00123, 0, 0, 0, -0.000493, -0.000986, 0.000563, 0.000204, -0.000611, 0.000265, 0, 0},
    {0, 0, 0.00194, 0.000422, 0.00117, 0.00059, -0.000173, -0.000138, -0.000237, 0, 0, 0, -0.00133, -0.00115, 0.00176, 6.87e-05, -0.00056, 0.000671, 0, 0},
    {0, 0, 0.0017, 0.000626, 0.000594, 0.000533, -0.000365, -0.000236, -0.000498, 0, 0, 0, 1.14e-05, -0.00108, 0.00245, 9.99e-05, -0.000882, 0.000735, 0, 0},
    {0, 0, 0.00178, 0.0007

In [18]:
if True:
    print(f"const double offsets_dca_y[{len(x_arrays)}][2] =" + " {")
    for iz in range(len(x_arrays)):
        if iz > 0: print(",")
        print("    {"+f"{(x_arrays[iz][0][4]+x_arrays[iz][0][5])/10000:.3g}"+f", {x_arrays[iz][0][15]/10000:.3g}", end="")
        print("}", end="")
    print("\n};")

const double offsets_dca_y[8][2] = {
    {0.0002, 0.000874},
    {-6.15e-06, 0.000121},
    {0.00048, -7.76e-05},
    {0.000846, 0.000204},
    {0.00176, 6.87e-05},
    {0.00113, 9.99e-05},
    {0.000575, 0.000101},
    {0.000126, -0.000209}
};


## Formula for Alignment Fit

The function used for fitting the mean shift as a function of azimuthal angle (φ) and longitudinal position (z) is:

<div align="center">

Δφ(z, φ) = ([0] + [1]·z)/10000 · sin(φ) + ([2] + [3]·z)/10000 · cos(φ) + [4]/1000

</div>

**where:**
- [0], [1], [2], [3], [4] are fit parameters extracted from the data,
- z is the longitudinal coordinate (e.g., along the beam axis),
- φ is the azimuthal angle.

---

### Why use this function for a 3D histogram fit?

- **Physical Motivation:**  
  Misalignments of detector layers often manifest as sinusoidal modulations in the measured hit positions as a function of φ, with possible linear dependence on z due to tilts or twists.

- **Parameter Extraction:**  
  Fitting this function to the 3D histogram of mean residuals (mean shift vs. φ and z) allows extraction of alignment parameters:
    - The coefficients of sin(φ) and cos(φ) (and their z-dependence) correspond to translations and tilts of the detector layer in the transverse plane.
    - The constant term can represent a global offset.

- **Usage:**  
  After fitting, the obtained parameters can be used to correct (align) the detector geometry, improving tracking and vertexing performance.